# Model Validation

Main goals:

* Use train/test/validation sets
* Use cross-validation
* Tune hyperparameters
* Visualise results

We will work with `sklearn` and the usual data science libraries that we import now:

In [ ]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import sklearn

# Spam or not spam?

The `spam.csv` file is described [here](https://www.kaggle.com/somesh24/spambase).

It contains emails classified as spam (class==1) and not spam (class==0). Each email also has some features associated with it, measuring the frequency of certain words and how many capital letters were used in the email.

Load the file into `pandas` and have a look at it. How many data points are there? Are the spam/not spam classes balanced?

In [ ]:
data = pd.read_csv('data/spam.csv')

print(f"There are {data.shape[0]} data points with {data.shape[1]} variables each")

print(data['class'].value_counts(normalize=True))

## Data partitioning

You will train a classifier for labeling email as spam or not. The k-nearest neighbours (KNN) classifier will need fine-tuning and you also want to be confident in the model's performance: the available data is slightly imbalanced and also potentially noisy.

Use `sklearn.model_selection.train_test_split` to first create a train/test split with appropriate proportions.

Then, split the test partition in half again to create test and validation sets.

Save the results in the variables `x_train`, `x_test`, `x_val`, `y_train`, `y_test`, `y_val`.

In [ ]:
from sklearn.model_selection import train_test_split

# x should be every column except the one we are trying to predict!
x = data.drop(['class'], axis=1)
# y is *only* the column we want to predict
y = data['class']

# Your code below...
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5)


Check the number of items in each partition: train/test/validation.

Make sure the x and y parts of each contain the same number of items.

In [ ]:
# Your code below...
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))
print(len(x_val), len(y_val))


`train_test_split` works on all sorts of data types: lists, `numpy` arrays, `pandas` Series and DataFrames. It returns the same type of data it was given.

Here, the `x` features are in a DataFrame, the `y` labels are in a Series.

How many examples of spam/not spam are in each partition?

(Because the `y` values are 0 and 1, you can use the `.sum()` method on a Series and divide by the length of it to get the proportion of positive classes. Or you can use the `.value_counts(normalize=True)` method - this is better when the values are not 0/1 but something like A/B.)

In [ ]:
# Your code here...

# Using .sum()
print(f"% spam in all data: {data['class'].sum()/len(data):.3f}")
print(f"% spam in training data: {y_train.sum()/len(y_train):.3f}")
print(f"% spam in test data: {y_test.sum()/len(y_test):.3f}")
print(f"% spam in validation data: {y_val.sum()/len(y_val):.3f}")

# One example of .value_counts()

print("Training data class proportions:")

print(y_train.value_counts(normalize=True))



## Train a model

Over the following steps, you will set up and train a model, then evaluate it.

First, create a `KNN` model (`model`) with the default settings.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
 
# Your code here...
model = KNN()


Now, use the model's `.fit()` method to learn the training data.

In [ ]:
# Your code here...
model.fit(x_train, y_train)


With the trained model, use its `.predict()` method to classify the test set. Save the result in the variable `predictions`.

Compare the first ten predictions to the expected values.

(Note: The output of the models is a `numpy` array, but the train/test data from `train_test_split` is in a `pandas.Series` object. You can get the raw values from a Series via the `.values` attribute.)  

In [ ]:
# Your code here...
predictions = model.predict(x_test)

print(predictions[0:10])
print(y_test[0:10].values)



Calculate the accuracy and F1 score for the model's predictions.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Your code here...
acc = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
              
print(f'Accuracy: {acc}')
print(f'F1: {f1}')


Use `sklearn.metrics.classification_report` to generate a report on the model's performance.

How does the model perform for each class? Would you rather a model which is better at classifying emails as spam, or as not spam?

In [ ]:
from sklearn.metrics import classification_report

# Your code here...
print(classification_report(y_test, predictions))

print('The model performs fairly well in general, but is better at identifying not-spam.')
print('So it will more often let spam through, and will also falsely flag real email as spam.')


## Cross-validation: confidence in model performance

How confident are you that the observed performance represents the "true" performance of the model? Could it just be a fluke of the way the training data was randomly split up? Perhaps the test examples were oddly similar to the training ones, or maybe too different?

Cross-validation will help you investigate this. We will use a non-exhaustive approach where we train on a few subsets of the data rather than every possible combination. Because cross-validation in `sklearn` automatically partitions data for us, we no longer need to worry about `train_test_split`.

We are going to use `sklearn.model_selection.cross_validate` to do all the work for us, but it needs a variety of parameters: a model, the data, a cross-validator, and some metrics for scoring.

First, set up a default `KNN` model as before (named `model`) and also a suitable cross-validator (named `my_cv`) with 5 folds. Given the data you are working with, should you use a stratified approach?

In [ ]:
# Your code here...
from sklearn.model_selection import StratifiedKFold

model = KNN()
my_cv = StratifiedKFold(n_splits=5)

print("Stratified is probably best here, given the class imbalance.")


## Cross-validation: choosing metrics

Next, we set up a dictionary of the metrics we want to use. The keys are our names for them, the values are strings which `sklearn` recognises and maps to metrics. Below, we've chosen useful ones for classification.

In [ ]:
my_metrics = {'accuracy': 'accuracy',
              'precision': 'precision_macro',
              'recall': 'recall_macro',
              'f1': 'f1_macro'}

## Cross-validation: putting it all together

Now, call the `cross_validate` function and pass it your model, the data (the `x` and `y` from the start), the cross-validator, and the metrics dictionary. Save the result in the output `scores`.

In [ ]:
from sklearn.model_selection import cross_validate

# Your code here...
scores = cross_validate(model, x, y, cv=my_cv, scoring=my_metrics)


## Cross-validation: the results

Print the mean and standard deviation of all the results across all the folds. (Tip: the data returned by `cross_validate` is all in `numpy` arrays, which have `.mean()` and `.std()` methods!)

How does this compare to the single model you trained before, using just one set of data?

In [ ]:
# Your code here...
for k, v in scores.items():
    print(f"Mean {k}: {v.mean():.4f}")
    print(f" (stdev = {v.std():.4f})")
    
print("The accuracy here is pretty similar to before, but p/r/f1 are all a little lower.")
print("This suggests we got a bit lucky with the first model.")


## Reporting performance

Now that we have some results in the `scores` variable, let's make them a bit more presentable. This makes it easier to share results with others.

We make a `pandas` DataFrame (`scores_df`) of the results and take a look.

In [ ]:
scores_df = pd.DataFrame(scores)
scores_df

Each row represents the results of one fold. There are six columns: the first two relate to how long the model took to train and to make predictions, while the others are the evaluation scores.

These scores are probably of more interest.

Let's drop the columns we don't need: this can be done by passing a list of column names to the `.drop()` method of a DataFrame and specifying `axis=1`, which is the column axis. `axis=0` is the rows.

In [ ]:
scores_df = scores_df.drop(['fit_time', 'score_time'], axis=1)

Finally, `seaborn` has a `barplot` function which can take this DataFrame and plot it. It will treat each row as an observation of a repeated experiment and automatically calculate and display the standard deviation for each column as error bars.

We call the `seaborn.barplot`  function and pass `scores_df` to it via the `data=` argument. Also include `ci='sd'` - otherwise `seaborn` will use a 95% confidence intervals instead of using the standard deviation of the data.

In [ ]:
sns.barplot(data=scores_df, ci='sd');

Plots in `seaborn` generally have sensible default values, but they are also highly configurable. This is usually done through a combination of initial argument settings, calling methods after the plot is created, or using `pyplot` to change specific things.

In [ ]:
# Changing the size
plt.figure(figsize=(10,4))

# Changing colours, order of the bars, etc
g = sns.barplot(data=scores_df, ci='sd', palette='pastel', capsize=0.1, order=['test_f1', 'test_precision', 'test_recall', 'test_accuracy']);

# Changing font sizes and adding labels
g.set_title('Model performance', fontsize=22)
g.set_xlabel('Metric')
g.set_ylabel('Score')
g.set_ylim(0.65, 0.9)

# Rotating labels - useful if they are too long and overlap.
plt.xticks(rotation=45);

# GridSearch: model tuning and cross-validation

Now, let's use grid search and cross-validation for fine-tuning the model. The `KNN` model has one hyperparameter we can adjust.

The GridSearchCV class needs a model, a dictionary of `{hyperparameters:values}` to try, a cross-validation object, and some metrics to use for evaluation.

First, create a default `KNN` model named `model`, as before. Then, create a `StratifiedKFold` object, with 5 folds, named `my_cv`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import StratifiedKFold

# Your code here...
model = KNN()
my_cv = StratifiedKFold(5)


## Grid search: hyperparameters

The `KNN` hyperparameter we want to investigate is called `n_neighbors` - this determines how many of the most similar data points are used to classify new data points.

Create a dictionary named `hyperparams` with one key, named after the hyperparameter we want to tune. The value for it should be a list with all the settings you want to try for that hyperparameter. Try the values 1 - 10 for this.

In [ ]:
# Your code here...
hyperparams = {"n_neighbors":[1,2,3,4,5,6,7,8,9,10]}


## Grid search: scoring

Knowing when the best hyperparameters have been found requires all models to be scored using some metric. `sklearn` will generally set some sensible default for this, but you can also choose your own. The ones available are below.

In [ ]:
print(sorted(sklearn.metrics.SCORERS.keys()))

Now create a dictionary named `my_metrics` of the metrics you want to use to evaluate the models with.

The keys are the names you use to identify the metrics. The values should be strings which `sklearn` recognises: use at least `accuracy` and `f1_macro`.

In [ ]:
# Your code here...
my_metrics = {'accuracy': 'accuracy',
              'precision': 'precision_macro',
              'recall': 'recall_macro',
              'f1': 'f1_macro'}


## Grid search: compile and fit

The final step is to create the `GridSearchCV` object and pass it all the arguments from above.

(Tip: Since we are using multiple metrics with `GridSearchCV`, we need to set `refit=False` because by default it tries to fit a best model overall once finished, but can only select the best model using a single metric, not multiple.)

Then, as with most things in `sklearn`, call the `.fit()` method.

It will chug along for a few seconds.

In [ ]:
from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(model, hyperparams, cv=my_cv, scoring=my_metrics, refit=False)

gridsearch.fit(x, y)

## Grid search: getting the results

The various statistics generated are stored in the `cv_results_` attribute of the `GridSearchCV` object you created. It's a big dictionary of data with scores per fold, per hyperparameter.  It also includes information about how long each model took to train and to evaluate.

Keys are descriptive strings of the data, while the values are arrays containing per-parameter scores and statistics. 

We used 10 settings for 1 hyperparameter, with 5 folds and 2+ metrics. Therefore, for each metric we have 5 items, each containing 10 items. There is also an additional key telling you the rank of each combination, so you can easily find the best one.

The cell below shows all the contents of this dictionary. Note that the metric names used in `my_metrics` are used as part of the names here - this could be useful for finding what you want.

In [ ]:
gridsearch.cv_results_

## Reporting fine-tuning results

That's a lot of data! But the most useful ones are those which give the mean scores per metric, per hyperparameter setting. For example `mean_test_f1` and `mean_test_accuracy` if you specified those metrics when setting up `GridSearchCV`.

Visualising this is a bit more fiddly, because the data is not in the ideal format for passing directly to `seaborn` if we want error bars shown.

Below, you can see how to create a DataFrame from the results and visualise the key components.

In [ ]:
# Turn GridSearch results into a dataframe
grid_results = pd.DataFrame(gridsearch.cv_results_)

# Drop all the junk
grid_results = grid_results.drop(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'params', 'mean_test_accuracy', 'std_test_accuracy',
       'rank_test_accuracy', 'mean_test_precision', 'std_test_precision',
       'rank_test_precision', 'mean_test_recall', 'std_test_recall', 'rank_test_recall',
       'mean_test_f1', 'std_test_f1', 'rank_test_f1'], axis=1)

# Melt the wide dataframe into a long one, using the hyperparameter value as the ID.
grid_results = grid_results.melt(id_vars='param_n_neighbors', var_name='metric')

# Remove digits in the metric names so that seaborn will group them for us
grid_results['metric'] = grid_results['metric'].str.replace(r'\d_', '_')
# Remove extra junk from metric names 
grid_results['metric'] = grid_results['metric'].str.replace(r'split_test_', '')

# Plot it
sns.catplot(data=grid_results, x='metric', y='value', hue='param_n_neighbors',kind='bar', ci='sd', height=5, aspect=2)

# Zoom in on the y-axis limit to show detail a bit better 
sns.mpl.pyplot.ylim(0.65, 0.85);

 # Getting the best model
 
The `GridSearchCV` above doesn't store the best model, because it used multiple scoring metrics. Now, repeat the process but only use `f1_macro` score.

In [ ]:
# Your code here...

model = KNN()
my_cv = StratifiedKFold(5)
hyperparams = {"n_neighbors":[1,2,3,4,5,6,7,8,9,10]}

gridsearch = GridSearchCV(model, hyperparams, cv=my_cv, scoring='f1_macro')
gridsearch.fit(x, y);

print(gridsearch.best_estimator_)



# Summary

* Data can be split into train/test/validation sets easily with `train_test_split`
* Cross-validation tools in `sklearn.model_selection` mean you can avoid doing that directly yourself
* Same with doing grid search: `GridSearchCV` handles everything
* Visualise results by reporting mean and std of scores across folds, per hyperparameter combination
* Extract the best model through the `best_estimator_` attribute of your grid search

# Going further

* With the information gained from the grid search, train a new model with the optimal settings
* Compare the results here to some kind of baseline model, to get a better idea of how good performance is
    * What data should you use to make your baselines?
* Try other models you are familiar with, which have additional hyperparameters you can tune
* Look into `sklearn.pipeline.Pipeline` for streamlining the process of testing multiple models and hyperparameters